In [1]:
from urllib.parse import urlparse
from urllib.parse import urldefrag
from urllib.request import urlopen
from file_storage import FileStorage
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import clear_output
from collections import defaultdict
import os
import numpy as np
from collections import Counter

In [2]:
def filter_url(url):
    bad_path_starts = [
        "/wiki/Wikipedia:",
        "/wiki/Help:",
        "/wiki/File:",
        "/wiki/Media:",
        "/wiki/MediaWiki:",
        "/wiki/MediaWiki_talk:",
        "/wiki/Talk:",
        "/wiki/Category:",
        "/wiki/Category_talk:",
        "/wiki/User:",
        "/wiki/User_talk:",
        "/wiki/Special:",
        "/wiki/Template:",
        "/wiki/Template_talk:",
        "/wiki/Wikipedia:",
        "/wiki/Wikipedia_talk:"
    ]
    path_is_ok = all([
        not urlparse(url).path.startswith(path_start)
        for path_start in bad_path_starts
    ])
    return path_is_ok and url.startswith("https://simple.wikipedia.org/wiki/")

def make_clear_data(data_from_first_homework_prefix, clear_data_prefix):
    first_homework_storage = FileStorage(data_from_first_homework_prefix, read_only=True)
    clear_storage = FileStorage(clear_data_prefix, read_only=False)

    count = first_homework_storage.count()
    for i, url in enumerate(first_homework_storage.dict.keys()):
        if i % 1000 == 0:
            clear_output()
            print("process {} / {}".format(i, count))
        if filter_url(url):
            html = first_homework_storage.read(url)
            clear_storage.write(url, html)

def extract_links_from_html(url, html):
    parser = BeautifulSoup(html)
    return [urljoin(url, link.get('href')) for link in parser.findAll('a')]

def get_L_M(storage):
    count = storage.count()
    L = defaultdict(lambda: [])
    M = defaultdict(lambda: [])
    for i, url in enumerate(storage.dict.keys()):
        if i % 100 == 0:
            clear_output()
            print("process {} / {}".format(i, count))
        html = storage.read(url)
        links = extract_links_from_html(url, html)
        for out_url in filter(filter_url, links):
            if out_url in storage.dict.keys():
                L[url].append(out_url)
                M[out_url].append(url)
    return L, M

In [3]:
CLEAR_STORAGE_PREFIX = "clear_storage"

In [4]:
make_clear_data("../task_1/hw_first_try", CLEAR_STORAGE_PREFIX)

process 221000 / 221610


In [5]:
clear_storage = FileStorage(CLEAR_STORAGE_PREFIX)

In [6]:
clear_storage.count()

174475

In [7]:
L, M = get_L_M(clear_storage)

process 174400 / 174475


In [8]:
def calculate_page_rank(L, delta, iterations):
    N = len(L)
    page_rank = {
        url: 1 / N
        for url in L
    }
    page_rank_vector = np.ones(N) / N
    for iterration in range(iterations):
        print("process {} / {}".format(iterration, iterations))
        new_page_rank = defaultdict(lambda: delta / N)
        for url in L:
            for out_url in L[url]:
                new_page_rank[out_url] += (1 - delta) * page_rank[url] / len(L[url])
        new_page_rank_vector = np.array([
            new_page_rank[url]
            for url in L
        ])
        new_page_rank_vector /= np.max(new_page_rank_vector)
        print("mean change:", np.mean(np.abs(new_page_rank_vector - page_rank_vector)))
        page_rank_vector = new_page_rank_vector
        for i, url in enumerate(new_page_rank):
            page_rank[url] = page_rank_vector[i]
    return page_rank

def calculate_page_rank_with_M(L, M, delta, iterations):
    N = len(L)
    page_rank = {
        url: 1 / N
        for url in L
    }
    page_rank_vector = np.ones(N) / N
    for iterration in range(iterations):
        print("process {} / {}".format(iterration, iterations))
        new_page_rank = {}
        for url in L:
            new_page_rank[url] = delta / N + (1 - delta) * sum([
                page_rank[out_url] / len(L[out_url])
                for out_url in M[url]
            ])
        new_page_rank_vector = np.array([
            new_page_rank[url]
            for url in L
        ])
        new_page_rank_vector /= np.max(new_page_rank_vector)
        print("mean change:", np.mean(np.abs(new_page_rank_vector - page_rank_vector)))
        page_rank_vector = new_page_rank_vector
        for i, url in enumerate(new_page_rank):
            page_rank[url] = page_rank_vector[i]
    return page_rank

In [9]:
page_rank = calculate_page_rank_with_M(L, M, 0.1, iterations=100)

process 0 / 100
mean change: 5.088897419085994e-05
process 1 / 100
mean change: 3.1585093616666614e-05
process 2 / 100
mean change: 2.126726322569772e-05
process 3 / 100
mean change: 1.17441561307464e-05
process 4 / 100
mean change: 6.939440103329714e-06
process 5 / 100
mean change: 4.558696272114575e-06
process 6 / 100
mean change: 3.2048473968067288e-06
process 7 / 100
mean change: 2.353242900803139e-06
process 8 / 100
mean change: 1.7816748397967782e-06
process 9 / 100
mean change: 1.384040675925267e-06
process 10 / 100
mean change: 1.0977249232722512e-06
process 11 / 100
mean change: 8.854806178484392e-07
process 12 / 100
mean change: 7.236883131510399e-07
process 13 / 100
mean change: 5.976271235805864e-07
process 14 / 100
mean change: 4.980153892897554e-07
process 15 / 100
mean change: 4.1847372017906833e-07
process 16 / 100
mean change: 3.5413509108846073e-07
process 17 / 100
mean change: 3.014155509881338e-07
process 18 / 100
mean change: 2.5782748857398067e-07
process 19 / 100

In [10]:
counter = Counter(page_rank)
for x, y in counter.most_common(n=20):
    print("{} {}".format(x, y))

https://simple.wikipedia.org/wiki/Main_Page 1.0
https://simple.wikipedia.org/wiki/International_Standard_Book_Number 0.10773110655310215
https://simple.wikipedia.org/wiki/United_States 0.060845932108152215
https://simple.wikipedia.org/wiki/List_of_Wikipedias 0.0466471570948122
https://simple.wikipedia.org/wiki/Definition 0.044995985661271304
https://simple.wikipedia.org/wiki/Country 0.03647917304012313
https://simple.wikipedia.org/wiki/United_Kingdom 0.03288250583963272
https://simple.wikipedia.org/wiki/English_language 0.03240051875595157
https://simple.wikipedia.org/wiki/Europe 0.03014627357971105
https://simple.wikipedia.org/wiki/France 0.029051856614157417
https://simple.wikipedia.org/wiki/Government 0.026824121884181897
https://simple.wikipedia.org/wiki/Germany 0.026322484424863023
https://simple.wikipedia.org/wiki/Mathematics 0.023202320058391436
https://simple.wikipedia.org/wiki/Law 0.022484686861608167
https://simple.wikipedia.org/wiki/Language 0.022234768641852563
https://simp

In [12]:
with open("page_rank.csv", "w") as handler:
    for x, y in counter.most_common():
        print("{},{}".format(x, y), file=handler)